In [1]:
import pymongo, re
import requests
import detectlanguage
from bs4 import BeautifulSoup
from deep_translator import GoogleTranslator

In [2]:
connectionStr = "mongodb+srv://<username>:<password>.tj5ah.mongodb.net/testDatabase?retryWrites=true&w=majority"

In [3]:
myClient = pymongo.MongoClient(connectionStr, serverSelectionTimeoutMS = 15000)

In [4]:
try:
    print(myClient.server_info())
except Exception:
    print("Unable to establish connection to the server!")

{'version': '4.4.10', 'gitVersion': '58971da1ef93435a9f62bf4708a81713def6e88c', 'modules': ['enterprise'], 'allocator': 'tcmalloc', 'javascriptEngine': 'mozjs', 'sysInfo': 'deprecated', 'versionArray': [4, 4, 10, 0], 'openssl': {'running': 'OpenSSL 1.0.1e-fips 11 Feb 2013', 'compiled': 'OpenSSL 1.0.1e-fips 11 Feb 2013'}, 'buildEnvironment': {'distmod': 'rhel70', 'distarch': 'x86_64', 'cc': '/opt/mongodbtoolchain/v3/bin/gcc: gcc (GCC) 8.5.0', 'ccflags': '-fno-omit-frame-pointer -fno-strict-aliasing -fasynchronous-unwind-tables -ggdb -pthread -Wall -Wsign-compare -Wno-unknown-pragmas -Winvalid-pch -Werror -O2 -Wno-unused-local-typedefs -Wno-unused-function -Wno-deprecated-declarations -Wno-unused-const-variable -Wno-unused-but-set-variable -Wno-missing-braces -fstack-protector-strong -fno-builtin-memcmp', 'cxx': '/opt/mongodbtoolchain/v3/bin/g++: g++ (GCC) 8.5.0', 'cxxflags': '-Woverloaded-virtual -Wno-maybe-uninitialized -fsized-deallocation -std=c++17', 'linkflags': '-pthread -Wl,-z,no

In [5]:
mydb = myClient.testDatabase

In [6]:
collectionName = input("Type in the name of the collection to access: ")

Type in the name of the collection to access: agrisDatasetCollection


In [7]:
myCollection = mydb[collectionName]

In [8]:
baseUrl = "https://agris.fao.org/agris-search/"

In [9]:
listOfAttrs = ["citation_title", "citation_author", "citation_publication_date", "citation_publisher", "citation_abstract"]

In [10]:
detectlanguage.configuration.api_key = "my_key"

In [11]:
fileName = input("Type in the name of the file to get the result URLs from: ")

Type in the name of the file to get the result URLs from: URLS_for_the_AGRIS_dataset


In [12]:
fullFileName = fileName + ".txt"

In [13]:
with open(fullFileName, "r") as file:
    for line in file:
        if len(line) != 0:
            resourceInfoDict = {}
            url = baseUrl + line.rstrip()
            page = requests.get(url)
            soup = BeautifulSoup(page.content, "html.parser")
            urlComponents = url.split("=")
            resourceInfoDict["_id"] = urlComponents[-1]
            counter = 0
            for attributeValue in listOfAttrs:
                metaElements = soup.find_all("meta", attrs = {"name": attributeValue})
                for metaElement in metaElements:
                    if attributeValue == "citation_abstract":
                        if (len(metaElement["content"]) < 5000) or (len(metaElement["content"]) == 0):
                            counter +=1
                            languageUsed = detectlanguage.detect(metaElement["content"])
                            if languageUsed != "en":
                                metaElement["content"] = GoogleTranslator(source='auto', target='en').translate(metaElement["content"])
                            metaElement["content"] = re.sub("<br />", " ", metaElement["content"])
                            metaElement["content"] = re.sub("<[^<]+?>", " ", metaElement["content"])
                            metaElement["content"] = re.sub(" +", " ", metaElement["content"])
                            metaElement["content"] = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", metaElement["content"])
                            metaElement["content"] = metaElement["content"].strip()
                        else:
                            metaElement["content"] = ""
                    resourceInfoDict[attributeValue] = metaElement["content"]
            myCollection.insert_one(resourceInfoDict)

In [14]:
file.close()